In [ ]:
import numpy as np
import pandas as pd

In [ ]:
path_raw0 = r"..\Raw" + "\\"
path_csv = r"..\CSV" + "\\"

file_station = r"..\Station Info.csv"

col_list = ["River", "Station", "Lon", "Lat", "Date", "Discharge"]

In [ ]:
df_station = pd.read_csv(file_station)
# df_station = df_station[df_station["quality"].isin(["High", "Medium"])]
# df_station.reset_index(drop=True, inplace=True)
print(df_station.shape)
df_station.head()

In [ ]:
for index, row in df_station.iterrows():
    if index % 100 == 0:
        print(index)
    folder = row["Folder"]
    station_id = row["GRDC_No"]
    file_in = f"{path_raw0}\\{folder}\\{station_id}_Q_Day.Cmd.txt"
    df = pd.read_csv(
        file_in, sep=";", skiprows=36, encoding="ansi", encoding_errors="ignore"
    )
    assert df.shape[0] > 0

    df.rename(columns={"YYYY-MM-DD": "Date", " Value": "Discharge"}, inplace=True)
    df["Date"] = pd.to_datetime(df["Date"])
    df.drop(columns=["hh:mm"], inplace=True)
    df.set_index("Date", inplace=True)
    df.replace(-999, np.nan, inplace=True)
    df.dropna(subset=["Discharge"], inplace=True)
    df = df.resample("D").mean()

    river = row["River"]
    station = row["Station"]

    # Drop some unreliable data
    if (river == "Indigirka") & (row["Station"] == "Vorontsovo"):
        df.loc["1995-07-01":"1995-09-30", "Discharge"] = np.nan
    elif (river == "Olenek") & (row["Station"] == "Sukhana"):
        df.loc["1996-09-21":"1996-09-30", "Discharge"] = np.nan

    df["River"] = river
    df["Station"] = station
    df["Lon"] = row["Lon"]
    df["Lat"] = row["Lat"]
    df.reset_index(drop=False, inplace=True)
    df = df[col_list]
    df.sort_values(by="Date", inplace=True)

    file_out = path_csv + f"{river}__{station}.csv"
    df.to_csv(file_out, index=False)